In [ ]:
import pandas as pd
import numpy as np
coilData =pd.read_csv('CoilHistory.csv')
operData =pd.read_csv('OperationsDetail.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#Select useful attributes with dictionary
coilDict = [0,2,3,4,5,6,8,9,11,14,15,16,20,21,31,32,33,34,36,37,38,39,40,41,44,45,46,48,49,50,51,52,53,54,56,57,58,59,
           61,62,63,64,65,66,68,69,70,71,73,74,75,76,77,78,79,80]
operDict  = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,25]
coil = coilData.iloc[:,coilDict]
oper = operData.iloc[:,operDict]

In [ ]:
#Part1: Missing Value Analysis
missing =coil.replace(" ",np.nan)
missing =missing.replace("  ",np.nan)
missing =missing.replace("   ",np.nan)
missing =missing.replace("    ",np.nan)
missing =missing.replace("     ",np.nan)
missing =missing.replace("      ",np.nan)
missing =missing.replace("       ",np.nan)
missing =missing.replace("        ",np.nan)

total_missing = missing.isnull().sum().sort_values(ascending = False)
percent_missing = (missing.isnull().sum()/missing.isnull().count()*100).sort_values(ascending = False)
missing4  = pd.concat([total_missing, percent_missing], axis=1, keys=['Total_Missing', 'Percent_Missing(%)'])
print("\n-----------Missing Value is--------------\n",missing4)

In [ ]:
#Drop the attributes which have more than 50% missing value
coil_1 = coil.drop(['INUCLS','INUNIT','INFNXO','INFNXU','INMAJL','INMINL','INSTRN','INTRST'], axis=1)
coil_1.head()

In [ ]:
#Transform Data Type
cols = coil_1.columns.tolist()
#Describe the variables
cat_vars = []
print ("\n the catagorical attributes are:")
for col in cols:
    if coil_1[col].dtype == "object":
        print (col)
        cat_vars.append(col)

#Use LableEncoder to transform data type
print ("\n Start transform attributes...")       
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

for col in cat_vars:
    tran = le.fit_transform(coil_1[col].tolist())
    tran_df = pd.DataFrame(tran, columns=['num_'+col])
    print("{col} is transformed into {num_col}".format(col=col,num_col='num_'+col))
#    print (le.classes_)
    coil_1 = pd.concat([coil_1, tran_df], axis=1)

In [ ]:
#Adjust for some attributes which should NOT be categorical variables
coil_2 = coil_1.drop(['num_INSTMS','num_INFTMS'], axis=1)
#coil_2[['INELAP']] = coil_2[['INELAP']].apply(pd.to_numeric)

#Add New Attribute
coil_2['yield loss']=(coil_2['INFGWT']-coil_2['INSLBS'])/coil_2['INSLBS']
coil_2.shape[1]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

#Types of Features
#Type of Machine
#1.Distribution of Some Selected Attributes
coil_2.loc[:,'INSNXO'].value_counts()
coil_2.loc[:,'INSRSQ'].value_counts()
coil_2.loc[:,'INSENT'].value_counts()
coil_2.loc[:,'INALLY'].value_counts()
coil_2.loc[:,'INDENS'].value_counts()


In [ ]:
coil_2.loc[:,'INHEAT'].value_counts()

In [ ]:
f,ax=plt.subplots(2,2,figsize=(20,15))
sns.countplot('INSWID',data=coil_2,ax=ax[0,0])
ax[0,0].set_title('No. of Steel Type')
sns.countplot('INSNXO',data=coil_2,ax=ax[0,1])
ax[0,1].set_title('No. of Operation Type')
sns.countplot('INALLY',data=coil_2,ax=ax[1,0])
ax[1,0].set_title('No. of Alloy Type')
sns.countplot('INSNXU',data=coil_2,ax=ax[1,1])
ax[1,1].set_title('No. of Next Unit')
plt.subplots_adjust(wspace=0.2,hspace=0.5)
plt.show()

In [ ]:
waitData=coil_2[coil_2['INSNXO']=="IN"]

In [ ]:
waitData['prev_time'] = waitData.groupby(['INVID#'])['INFTMS'].shift(1)
waitData['prev_time']

In [ ]:
a=[]
for i in range(len(waitData)):
        if pd.isnull(waitData["prev_time"][waitData.index[i]])==False:
           b =waitData.index[i]-waitData.index[i-1]-1
        else:
            b=0
        a.append(b)
waitData['count_steps']=a          

In [ ]:
coil_2.columns

In [ ]:
#waitData[['INSENT','INSNXO','count_steps']]
coil_2[["INSGWT","INFGWT","yield loss"]]

In [ ]:
import datetime

def interval_seconds(prev,start):
    if pd.isnull(prev)==False:
        prev=str(prev)
        prev=prev[0:19]
        start=str(start)
        start=start[0:19]
        prev=datetime.datetime.strptime(prev, '%Y-%m-%d %H:%M:%S')
        start=datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
        i=start-prev
        i=i.seconds
    else:
        i=0
    return(i)

def interval_days(prev,start):
    if pd.isnull(prev)==False:
        prev=str(prev)
        prev=prev[0:19]
        start=str(start)
        start=start[0:19]
        prev=datetime.datetime.strptime(prev, '%Y-%m-%d %H:%M:%S')
        start=datetime.datetime.strptime(start, '%Y-%m-%d %H:%M:%S')
        i=start-prev
        i=i.days
    else:
        i=0
    return(i)

waitData['w1'] = waitData.apply(lambda x: interval_seconds(x.prev_time,x.INSTMS), axis = 1).to_frame()
waitData['w2'] = waitData.apply(lambda x: interval_days(x.prev_time,x.INSTMS), axis = 1).to_frame()
waitData['waitTime']=waitData['w1']+waitData['w2']*86400

waitData= waitData.drop(['w1','w2'], axis=1)

In [ ]:
#Use hour as waitTime unit
waitData['waitTime']=waitData['waitTime']/3600

In [ ]:
#-------Step1: correlation between waiting time and attributes--------
corrData = waitData[['waitTime','num_INTYPE','INDENS','INHEAT','IN#PAS','INALLY','num_INALLY','num_INVCLK','num_INSNXO','num_INSACT','INSWID','INSGAG','INSLEN','INSGWT','num_INFACT','num_INFSTA','INFGWT','INELAP','yield loss']]
sns.heatmap(corrData.corr(),annot=True,cmap='RdYlGn',linewidths=0.2) #data.corr()-->correlation matrix
fig=plt.gcf()
fig.set_size_inches(10,8)
plt.show()

In [ ]:
#----Step2: the waitTime VS. attributes------
#1. Width
waitData['INSWID'].describe()

In [ ]:
waitData['waitTime'].describe()

In [ ]:
f,ax=plt.subplots(1,3,figsize=(20,8))
temp=waitData['waitTime']
sns.distplot(waitData[waitData['waitTime']<=temp.mean(axis=0)].INSWID,ax=ax[0])
ax[0].set_title('width distribution in Short waitTime')

sns.distplot(waitData[waitData['waitTime']>temp.mean(axis=0)].INSWID,ax=ax[1])
ax[1].set_title('width distribution in Long waitTime')

sns.distplot(waitData.INSWID,ax=ax[2])
ax[2].set_title('width distribution')
plt.show()

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
waitData["waitTime_tooLong"]= (waitData['waitTime']>=1.822699e+05)
sns.violinplot("INSWID","INSGWT", hue="waitTime_tooLong", data=waitData,split=True,ax=ax[0])
ax[0].set_title('width and weight vs whether wait too long')
ax[0].set_yticks(range(0,55,5))
sns.violinplot("INSLEN","INSGAG", hue="waitTime_tooLong", data=waitData,split=True,ax=ax[1])
ax[1].set_title('length and Gauge vs whether wait too long')
#ax[1].set_yticks(range(0,0.8,0.1))
plt.show()

In [ ]:
#2. Gauge
waitData['INSGAG'].describe()

In [ ]:
f,ax=plt.subplots(1,3,figsize=(20,8))
sns.distplot(waitData[waitData['INSGAG']<=0.078261].INSGAG,ax=ax[0])
ax[0].set_title('waitTime in Small Gauge')
sns.distplot(waitData[waitData['INSGAG']>0.078261].INSGAG,ax=ax[1])
ax[1].set_title('waitTime in Large Gauge')
sns.distplot(waitData.INSGAG,ax=ax[2])
ax[2].set_title('waitTime in All Gauge')
plt.show()

In [ ]:
#-------Step3:Feature Engineering------------
##---------Transform Categorical Variable--------###
pick_x_cat =[4,7,12,13,14,15,24,29,36,42]
pick_x=[2,6,10,16,17,18,21,22,25,26,30,33,34,35,39,40,43,44,47,71,73]+pick_x_cat
pick_y=[74]                                                  

waitData=waitData.dropna() 
x = waitData.iloc[:,pick_x]
y = waitData.iloc[:,pick_y]

In [ ]:
#Make Sankey Diagram
x.to_csv("x_sankey.csv",index=False,sep=',')
y.to_csv("y_sankey.csv",index=False,sep=',')

In [ ]:
##-----------Method1:Target Encoding-----------
from sklearn.base import BaseEstimator, TransformerMixin
class TargetEncoder(BaseEstimator, TransformerMixin):
    """Target encoder.
    
    Replaces categorical column(s) with the mean target value for
    each category.

    """
    
    def __init__(self, cols=None):
        """Target encoder
        
        Parameters
        ----------
        cols : list of str
            Columns to target encode.  Default is to target 
            encode all categorical columns in the DataFrame.
        """
        if isinstance(cols, str):
            self.cols = [cols]
        else:
            self.cols = cols
        
        
    def fit(self, X, y):
        """Fit target encoder to X and y
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values.
            
        Returns
        -------
        self : encoder
            Returns self.
        """
        
        # Encode all categorical cols by default
        if self.cols is None:
            self.cols = [col for col in X 
                         if str(X[col].dtype)=='object']

        # Check columns are in X
        for col in self.cols:
            if col not in X:
                raise ValueError('Column \''+col+'\' not in X')

        # Encode each element of each column
        self.maps = dict() #dict to store map for each column
        for col in self.cols:
            tmap = dict()
            uniques = X[col].unique()
            for unique in uniques:
                tmap[unique] = y[X[col]==unique].mean()
            self.maps[col] = tmap
            
        return self

        
    def transform(self, X, y=None):
        """Perform the target encoding transformation.
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
            
        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """
        Xo = X.copy()
        for col, tmap in self.maps.items():
            vals = np.full(X.shape[0], np.nan)
            for val, mean_target in tmap.items():
                vals[X[col]==val] = mean_target
            Xo[col] = vals
        return Xo
            
            
    def fit_transform(self, X, y=None):
        """Fit and transform the data via target encoding.
        
        Parameters
        ----------
        X : pandas DataFrame, shape [n_samples, n_columns]
            DataFrame containing columns to encode
        y : pandas Series, shape = [n_samples]
            Target values (required!).

        Returns
        -------
        pandas DataFrame
            Input DataFrame with transformed columns
        """
        return self.fit(X, y).transform(X, y)

In [ ]:
# Target encode the categorical data
te = TargetEncoder()
x_target_encoded = te.fit_transform(x, y)
x_target_encoded.shape[0]

In [ ]:
#according to attribute meaning --> INALLOY divided according to the first digit;
#x_target_encoded["INALLY"]=str(x["INALLY"])[0]
x_target_encoded['INALLY']=x['INALLY'].apply(lambda x: x[0])

In [ ]:
x_target_encoded["INALLY"]=x_target_encoded["INALLY"].replace("U",10)
x_target_encoded["INALLY"]=x_target_encoded["INALLY"].replace("K",11)
x_target_encoded["INALLY"]=x_target_encoded["INALLY"].replace("C",12)
x_target_encoded["INALLY"]=x_target_encoded["INALLY"].replace("T",13)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import RidgeCV, LassoCV, Ridge, Lasso

#Embeded Method:
reg = LassoCV()
reg.fit(x_target_encoded, y)
print("Best alpha using built-in LassoCV: %f" % reg.alpha_)
print("Best score using built-in LassoCV: %f" %reg.score(x_target_encoded, y))
coef = pd.Series(reg.coef_, index = x.columns)

In [ ]:
print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")

In [ ]:
imp_coef = coef.sort_values()
import matplotlib
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Feature importance using Lasso Model")

In [ ]:
corrData = pd.concat( [x_target_encoded,y], axis=1 )
corrData

In [ ]:
#Filter Method
#Using Pearson Correlation
corrData = pd.concat( [x_target_encoded,y], axis=1 )
plt.figure(figsize=(12,10))
cor = corrData.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

In [ ]:
#Correlation with output variable
cor_target = abs(cor["waitTime"])
#Selecting highly correlated features
relevant_features = cor_target[cor_target>0.4]
relevant_features

In [ ]:
#Wrapper Method
#Feed the features to the selected Machine Learning algorithm and based on the 
#model performance you add/remove the features.
#i. Backward Elimination
#Adding constant column of ones, mandatory for sm.OLS model
import statsmodels.api as sm

#Backward Elimination
cols = list(x_target_encoded.columns)
pmax = 1
while (len(cols)>0):
    p= []
    x_new = x_target_encoded[cols]
    #x_new = sm.add_constant(x_new)
    model = sm.OLS(y,x_new.astype(float)).fit()
    p = pd.Series(model.pvalues.values[0:],index = cols)      
    pmax = max(p)
    feature_with_p_max = p.idxmax()
    if(pmax>0.05):
        cols.remove(feature_with_p_max)
    else:
        break
selected_features_BE = cols
print(selected_features_BE)


In [ ]:
# RFE (Recursive Feature Elimination)
#no of features
nof_list=np.arange(1,13)            
high_score=0
#Variable to store the optimum features
nof=0           
score_list =[]
for n in range(len(nof_list)):
    X_train, X_test, y_train, y_test = train_test_split(x_target_encoded,y, test_size = 0.3, random_state = 0)
    model = LinearRegression()
    rfe = RFE(model,nof_list[n])
    X_train_rfe = rfe.fit_transform(X_train,y_train)
    X_test_rfe = rfe.transform(X_test)
    model.fit(X_train_rfe,y_train)
    score = model.score(X_test_rfe,y_test)
    score_list.append(score)
    if(score>high_score):
        high_score = score
        nof = nof_list[n]
print("Optimum number of features: %d" %nof)
print("Score with %d features: %f" % (nof, high_score))

In [ ]:
rfe = RFE(model, 5)
#Transforming data using RFE
X_rfe = rfe.fit_transform(x_target_encoded,y)  
#Fitting the data to model
model.fit(X_rfe,y)
print(rfe.support_)
print(rfe.ranking_)

In [ ]:
x_target_encoded.columns #'INDENS', 'INSGAG',INFGAG','INSPCS', 'INFPCS'

In [ ]:
#Conlusion: we will select:
#INDENS INSGAG INSPCS INFAG INFPCS
#INSXNU INFACT INHEAT
#INSSCL IN#PAS
#INALLY

In [ ]:
x_sel =[0,1,4,9,10,12,17,20,21,22,23,26]
x_selected = x_target_encoded.iloc[:,x_sel]
x_selected

In [ ]:
#Save files for further use
x_selected.to_csv("x_selected.csv",index=False,sep=',')
y.to_csv("y.csv",index=False,sep=',')

In [ ]:
import pandas as pd
x_selected =pd.read_csv('x_selected.csv')
y =pd.read_csv('y.csv')

In [ ]:
#####----------Build ML Model----------###########
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold, train_test_split
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

#------<Model1: LightGBM>----------
data_prepared = x_selected.astype(float)
data_labels = y.astype(int)

# Split into training and testing data
train_features, test_features, train_labels, test_labels = train_test_split(data_prepared, data_labels,random_state = 42)
get_ipython().system('pip install lightgbm')
import lightgbm as lgb

#Evaluation of the Model
# Visualization
plt.rcParams['font.size'] = 18
get_ipython().run_line_magic('matplotlib', 'inline')

# Governing choices for search
N_FOLDS = 5
MAX_EVALS = 5


In [ ]:
# First we can create a model with the default value of hyperparameters and score it using cross validation with early stopping. Using the cv LightGBM function requires creating a Dataset.
# Note: here we use **Automated Hyperparameter Tuning**
#Establish a baseline model
from sklearn.model_selection import cross_val_score

base_LGBM = lgb.LGBMRegressor(random_state=50)
# Training set
train_set = lgb.Dataset(train_features, label = train_labels)
test_set = lgb.Dataset(test_features, label = test_labels)

# Default hyperparamters
hyperparameters = base_LGBM.get_params()
baseline_error = cross_val_score(base_LGBM, train_features, train_labels, scoring='neg_mean_squared_error', cv=10)
baseline_MSE_mean= baseline_error.mean()
print('The MSE for LGBM is:', baseline_MSE_mean)


# Train and make predicions with model
base_LGBM.fit(train_features, train_labels)
y_pred = base_LGBM.predict(test_features)

from sklearn.metrics import mean_squared_error
MSE_Test=mean_squared_error(test_labels, y_pred)
print('The baseline model scores {:.5f} RSE on the test set.'.format(MSE_Test))

In [ ]:
hyperparameters

In [ ]:
#Tune the parameter
#----Hyper Parameter Tuning: Use GridSearchCV--------------
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
import math
def lightGBM_model(X, y):
    gsc = GridSearchCV(
        estimator=lgb.LGBMRegressor(random_state=50),
        param_grid={
            'learning_rate': [0.01],
            'num_leaves': [24,50,80],
            'max_depth': [-1],
            'max_bin':[20,90],
            'min_child_samples': [20,30],
            'min_child_weight':[0.001],
            'n_estimators':[100,200],
            'subsample': [0.01, 1.0] ,
            'subsample_for_bin':[100000,200000],
            'reg_alpha':[0.0],
            'reg_lambda':[0.0]
        },
        cv=10, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

    grid_result = gsc.fit(X, y)
    best_params = grid_result.best_params_
    best_lightGBM = lgb.LGBMRegressor(num_leaves=best_params["num_leaves"], max_bin=best_params["max_bin"], 
                                      min_child_samples=best_params["min_child_samples"],
                                     n_estimators=best_params["n_estimators"],
                                      subsample=best_params["subsample"],
                                      subsample_for_bin=best_params["subsample_for_bin"])

    scoring = {
               'abs_error': 'neg_mean_absolute_error',
               'squared_error': 'neg_mean_squared_error',
                "r2":"r2"                               }

    scores = cross_validate(best_lightGBM, X, y, cv=10, scoring=scoring, return_train_score=True)
    return "MAE :", abs(scores['test_abs_error'].mean()), "| RMSE :", math.sqrt(abs(scores['test_squared_error'].mean())),"| r2 :", scores['test_r2'].mean()

In [ ]:
#
X=data_prepared
y=data_labels
lightGBM_model(X,y)

In [ ]:
#Mode2: Random Forest
#---------Divide training and testing data
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
#or use pipeline function to get the same purpose.
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                        RandomForestRegressor(n_estimators=100))
hyperparameters = { 'randomforestregressor__max_features' : ['auto', 'sqrt', 'log2'],
                  'randomforestregressor__max_depth': [None, 5, 3, 1]}
#------cross validation---------
clf = GridSearchCV(pipeline, hyperparameters, cv=10)
y_train_new=y_train/3600
y_test_new=y_test/3600
clf.fit(X_train, y_train_new)
y_pred_new = clf.predict(X_test)
#-------cv=10------------
print(mean_squared_error(y_test_new, y_pred_new))
print(r2_score(y_test_new, y_pred_new))
print(mean_squared_error(y_test_new, y_pred_new))

In [ ]:
#--------Model3.Using Neural Network Model
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)
scaler = preprocessing.StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

y_test_new=y_test/3600
y_train_new=y_train/3600

from keras.models import Sequential
from keras.layers import Dense

model=Sequential()


model.add(Dense(64,activation='relu',input_dim=X_train_scaled.shape[1]))
model.add(Dense(1))
model.compile(loss='mse',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train_scaled,y_train_new,batch_size=32,epochs=10,verbose=1,validation_data=(X_test_scaled,y_test_new))
model.compile(loss='mse',optimizer='rmsprop',metrics=['accuracy'])
model.fit(X_train_scaled,y_train_new,batch_size=32,epochs=10,verbose=1,validation_data=(X_test_scaled,y_test_new))
print(mean_squared_error(y_test_new, y_pred_new))
print (clf.best_params_)
print(r2_score(y_test_new, y_pred_new))
print(mean_squared_error(y_test_new, y_pred_new))

In [ ]:
#--------Model4.Using KNeighborsRegressor-----------
#n_neighbors: the number of neighbors, is set to 5
#algorithm: for computing nearest neighbors, is set to auto
#p: set to 2, corresponding to Euclidean distance

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)
knn = KNeighborsRegressor()

hyperparameters = {'n_neighbors' : [7,8,9,10], 'algorithm' : ['auto', 'brute', 'ball_tree','kd_tree'],
                  'weights': ['uniform','distance']}

clf = GridSearchCV(knn, hyperparameters)
y_train_new=y_train/3600
y_test_new=y_test/3600
clf.fit(X_train, y_train_new)
y_pred_new = clf.predict(X_test)

In [ ]:
# best KNN
print (clf.best_params_)
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.externals import joblib

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
import math

y_new=y/3600

best_knn = KNeighborsRegressor(algorithm='brute', n_neighbors=9, weights='uniform')
scores = cross_validate(best_knn, X, y_new, cv=10, scoring={'abs_error': 'neg_mean_absolute_error','squared_error': 'neg_mean_squared_error',"r2":"r2"}, return_train_score=True)
# evaluations
print(abs(scores['test_abs_error'].mean()))
print(math.sqrt(abs(scores['test_squared_error'].mean())))
print(scores['test_r2'].mean())

In [ ]:
#----------Model5: Linear Regression--------------
from sklearn import linear_model
regr = linear_model.LinearRegression()
y_test_new=y_test/3600
y_train_new=y_train/3600
regr.fit(X_train, y_train_new)
LinearRegression(fit_intercept=True,normalize=False,copy_X=True,n_jobs=1)
print(regr.coef_) 
print(regr.intercept_)
y_pred_new = regr.predict(X_test)
print(y_pred_new)
print(mean_absolute_error(y_test_new,y_pred_new))
print(r2_score(y_test_new, y_pred_new))
print(mean_squared_error(y_test_new, y_pred_new))
print(math.sqrt(mean_squared_error(y_test_new, y_pred_new)))
hyperparameters = regr.get_params()
hyperparameters
y_pred_new = regr.predict(X_test_std)
print(y_pred_new)
from sklearn.metrics import mean_absolute_error
print(mean_absolute_error(y_test_new,y_pred_new))
print(r2_score(y_test_new, y_pred_new))
print(math.sqrt(mean_squared_error(y_test_new, y_pred_new)))#------ no significant difference
regr2 = linear_model.LinearRegression(copy_X=True, fit_intercept=True, n_jobs=2, normalize=True)
regr2.fit(X_train_std, y_train_new)
print(regr2.coef_) 
y_pred_new = regr2.predict(X_test_std)
print(y_pred_new)
print(r2_score(y_test_new, y_pred_new))
print(mean_squared_error(y_test_new, y_pred_new))
print(math.sqrt(mean_squared_error(y_test_new, y_pred_new)))

In [ ]:
#-------Model6: Bayesianridge-----------
from sklearn.linear_model import BayesianRidge
reg = BayesianRidge()
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)
reg.fit(X_train, y_train_new)

In [ ]:
y_pred_new = reg.predict(X_test)
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import BayesianRidge
reg_br = BayesianRidge()
hyperparameters = {'alpha_1' : [1e-06,2e-06,3e-03,4e-01], 'alpha_2' : [1e-06, 0.1, 1,10],
                  'lambda_1': [1e-06,2e-06,3e-03,4e-01],'lambda_2':[1e-06, 0.1, 1,10]}

clf = GridSearchCV(reg_br, hyperparameters, cv=10, scoring='neg_mean_squared_error')
clf.fit(X, y_new)
print (clf.best_params_)#best bayesianridge

best_br= BayesianRidge(alpha_1=1e-06, alpha_2= 10, lambda_1= 0.4, lambda_2= 1e-06)
best_br.fit(X_train, y_train_new)
y_pred_new = best_br.predict(X_test)


print(mean_absolute_error(y_test_new,y_pred_new))
print(r2_score(y_test_new, y_pred_new))
print(math.sqrt(mean_squared_error(y_test_new, y_pred_new)))